In [1]:
import networkx as nx
import freeman as fm

In [2]:
g = fm.load('marvel.gml')

In [3]:
g.label_nodes('name')
g.set_all_nodes(size=15, labpos='hover')
g.set_all_edges(color=(0, 0, 0, 0.5))

In [4]:
g.draw()

In [5]:
ec = nx.eigenvector_centrality(g, max_iter=10000)

In [6]:
import pandas as pd
data = pd.DataFrame({
    'id' : [i for i in g.nodes],
    'Name': [g.nodes[i]['name'] for i in g.nodes],
    'eigenvector': [i for i in ec.values()]
})

In [7]:
data.head()

,id,Name,eigenvector
0,1009220,Captain America,2.898576e-27
1,1010740,Winter Soldier,2.898576e-27
2,1009471,Nick Fury,8.214196e-19
3,1009552,S.H.I.E.L.D.,8.214196e-19
4,1009228,Sharon Carter,8.214196e-19


In [8]:
char_infos = pd.read_csv('dataset/charcters_stats.csv', index_col=0)
char_infos.head()

,Alignment,Intelligence,Strength,Speed,Durability,Power,Combat,Total
Name,,,,,,,,
3-D Man,good,50,31,43,32,25,52,233
A-Bomb,good,38,100,17,80,17,64,316
Abe Sapien,good,88,14,35,42,35,85,299
Abin Sur,good,50,90,53,64,84,65,406
Abomination,bad,63,80,53,90,55,95,436


In [9]:
semi_df = pd.merge(data, char_infos, how='inner', on=['Name']) 

In [10]:
semi_df

,id,Name,eigenvector,Alignment,Intelligence,Strength,Speed,Durability,Power,Combat,Total
0,1009220,Captain America,2.898576e-27,good,63,19,35,56,46,100,319
1,1010740,Winter Soldier,2.898576e-27,good,56,32,35,65,60,84,332
2,1009471,Nick Fury,8.214196e-19,good,75,11,23,42,25,100,276
3,1009515,Punisher,2.898576e-27,good,50,16,23,28,22,100,239
4,1009535,Red Skull,8.214196e-19,bad,75,10,12,14,19,80,210
...,...,...,...,...,...,...,...,...,...,...,...
172,1009322,Goblin Queen,2.898576e-27,bad,75,10,23,28,65,56,257
173,1009217,Callisto,1.029781e-41,bad,63,53,23,42,45,85,311
174,1009150,Agent Zero,1.029781e-41,good,75,28,38,80,72,95,388
175,1009160,Arclight,2.898576e-27,bad,38,63,23,42,52,70,288


In [11]:
# Por enquanto não estamos usando os poderes
# powers = pd.read_csv('dataset/superheroes_power_matrix.csv')
# complete_df = pd.merge(semi_df, powers, how='inner', on=['Name'])
# complete_df

In [12]:
gender_values = semi_df['Gender'].unique()
for v in gender_values[:-1]:
    semi_df[v] = [1 if i == v else 0 for i in semi_df['Gender']]

KeyError: 'Gender'

In [ ]:
#adicionando a raça dos personagens
semi_df.loc[semi_df["Name"] == "Red Skull", ["Race"]] = "Human"
semi_df.loc[semi_df["Name"] == "Emma Frost", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Rogue", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Omega Red", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Machine Man", ["Race"]] = "Cyborg"
semi_df.loc[semi_df["Name"] == "Nightcrawler", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Namor", ["Race"]] = "Atlantean"
semi_df.loc[semi_df["Name"] == "Cloak", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Dagger", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Cypher", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Skaar", ["Race"]] = "Human / Radiation"
semi_df.loc[semi_df["Name"] == "Dormammu", ["Race"]] = "Cosmic Entity"
semi_df.loc[semi_df["Name"] == "Mephisto", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Ego", ["Race"]] = "God / Eternal"
semi_df.loc[semi_df["Name"] == "Man-Thing", ["Race"]] = "Human / Altered"
semi_df.loc[semi_df["Name"] == "Clea", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Wonder Man", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Shadow King", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Hope Summers", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Shatterstar", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Leader", ["Race"]] = "Human / Radiation"
semi_df.loc[semi_df["Name"] == "Wolfsbane", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Forge", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Kang", ["Race"]] = "Human"
semi_df.loc[semi_df["Name"] == "Multiple Man", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Cecilia Reyes", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Northstar", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Warbird", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Sage", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Songbird", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Hydro-Man", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Genesis", ["Race"]] = "Mutant / Clone"
semi_df.loc[semi_df["Name"] == "Blizzard", ["Race"]] = "Inhuman"
semi_df.loc[semi_df["Name"] == "Blob", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "X-Man", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Cannonball", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Mimic", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Lady Bullseye", ["Race"]] = "Human"
semi_df.loc[semi_df["Name"] == "Namora", ["Race"]] = "Atlantean"
semi_df.loc[semi_df["Name"] == "Vindicator", ["Race"]] = "Human"
semi_df.loc[semi_df["Name"] == "Snowbird", ["Race"]] = "Demi-God"
semi_df.loc[semi_df["Name"] == "Ares", ["Race"]] = "God / Eternal"
semi_df.loc[semi_df["Name"] == "Moonstone", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Box", ["Race"]] = "Human"
semi_df.loc[semi_df["Name"] == "Meteorite", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Annihilus", ["Race"]] = "Alien"
semi_df.loc[semi_df["Name"] == "Man-Wolf", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Tempest", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Crimson Dynamo", ["Race"]] = "Human"
semi_df.loc[semi_df["Name"] == "Chameleon", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Namorita", ["Race"]] = "Mutant / Clone"
semi_df.loc[semi_df["Name"] == "John Wraith", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Thundra", ["Race"]] = "Human / Altered"
semi_df.loc[semi_df["Name"] == "Jigsaw", ["Race"]] = "Human"
semi_df.loc[semi_df["Name"] == "Firebird", ["Race"]] = "Human / Radiation"
semi_df.loc[semi_df["Name"] == "Firelord", ["Race"]] = "Alien"
semi_df.loc[semi_df["Name"] == "Tinkerer", ["Race"]] = "Human"
semi_df.loc[semi_df["Name"] == "Pyro", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Molten Man", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Feral", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Synch", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Meltdown", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Franklin Storm", ["Race"]] = "Human"
semi_df.loc[semi_df["Name"] == "Boomer", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Lyja", ["Race"]] = "Alien"
semi_df.loc[semi_df["Name"] == "Corsair", ["Race"]] = "Human"
semi_df.loc[semi_df["Name"] == "Morph", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Stacy X", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Silverclaw", ["Race"]] = "Demi-God"
semi_df.loc[semi_df["Name"] == "Beak", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Morlun", ["Race"]] = "Vampire"
semi_df.loc[semi_df["Name"] == "Stardust", ["Race"]] = "Alien"
semi_df.loc[semi_df["Name"] == "Goblin Queen", ["Race"]] = "Mutant / Clone"
semi_df.loc[semi_df["Name"] == "Callisto", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Agent Zero", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Yellow Claw", ["Race"]] = "Human"
semi_df.loc[semi_df["Name"] == "Shriek", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Arclight", ["Race"]] = "Mutant / Clone"
semi_df.loc[semi_df["Name"] == "Leech", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Fabian Cortez", ["Race"]] = "Mutant"

In [ ]:
race_values = semi_df['Race'].unique()
for v in race_values[:-1]:
    semi_df[v] = [1 if i == v else 0 for i in semi_df['Race']]

In [ ]:
semi_df = semi_df.drop_duplicates(subset=['Name'], keep='first')

In [ ]:
semi_df['Alignment'].value_counts()

In [ ]:
semi_df = semi_df[semi_df['Alignment'] != "neutral"]

semi_df['Alignment'] = semi_df['Alignment'].replace('good', 1)
semi_df['Alignment'] = semi_df['Alignment'].replace('bad', 0)

In [ ]:
import statsmodels.api as sm

# Linear Regression

indep_variables = []
for i in gender_values[:-1]:
    indep_variables.append(i)
for i in race_values[:-1]:
    indep_variables.append(i)
indep_variables.append('eigenvector')

model = sm.OLS(semi_df['Alignment'], semi_df[[i for i in indep_variables]])
result = model.fit()
result.summary()

In [ ]:
# Logistic Regression

model = sm.Logit(semi_df['Alignment'], semi_df[[i for i in indep_variables]])
result = model.fit(method='ncg')
result.summary()